# Metric Benchmarking

## Defining Metrics

In [94]:
import os
import pandas as pd
import logging

# Assuming other imports like tqdm are used elsewhere in your code


def get_all_human_ncbi_gene_ids():
    """Get all human NCBI gene IDs
    Returns:
        - gene_ids (list): list of gene IDs as integers
    """
    data_path = os.path.join("..", "data", "Ensembl", "ensembl.human.csv")

    # Open DataFrame
    df = pd.read_csv(data_path)

    # Ensure NCBI gene IDs are integers. NaN values are ignored by dropna before conversion.
    gene_ids = df["NCBI gene (formerly Entrezgene) ID"].dropna().astype(int).to_list()

    # Removing duplicates and sorting can be done using `sorted(set(...))` for unique and ordered IDs
    gene_ids = sorted(set(gene_ids))

    logging.info(f"Number of Human genes: {len(gene_ids)}")

    return gene_ids


# Example usage
human_gene_ids = get_all_human_ncbi_gene_ids()

2024-01-31 17:24:59,242 - INFO - Number of Human genes: 27338


### iLINCS: Extreme Correlation Signed Log P-value Top 100 up- & -down

In [29]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, sys, os
import logging
from scipy.stats import norm

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

In [128]:
def compute_signed_significance(p_values, log2fcs, genes):
    """
    Calculate signed significance for each gene in the signature.
    Args:
    - p_values (list): list of p-values
    - log2fcs (list): list of log2fc values
    - genes (list): list of gene ids

    Returns:
    - signed_significance (list): [(gene_id, ss), ...]
    """
    signed_significance = list()
    for p_value, log2fc, gene in zip(p_values, log2fcs, genes):
        # Handle cases where AdjPValue is nan values
        # if (row["AdjPValue"] != 0) and (not np.isnan(row["AdjPValue"])):
        if not np.isnan(p_value):
            sign = np.sign(log2fc)
            ss = sign * (-np.log10(p_value))
            signed_significance.append((gene, ss))
    return signed_significance


def compute_extreme_signed_significance(signed_significance, k=100):
    """
    Compute Extreme Signed Significance for top & bottom k ss genes, rest
    are assigned value of 0.

    Arguments:
    - signed_significance: list of tupples
        List of tupples [(geneid,ss), ...]
    - k: int()
        Top/Bottom k genes.

    Returns:
    - extreme_signed_significance: list of tupples
        List of gene ids and signed significance
    """
    # sort signed significance by value
    sorted_signed_significance = sorted(
        signed_significance, key=lambda x: x[1], reverse=True
    )

    # get top_bottom values
    top_bottom_genes = [ss[0] for ss in sorted_signed_significance[:k]] + [
        ss[0] for ss in sorted_signed_significance[-k:]
    ]

    # set values for non top/bottom k to 0
    extreme_signed_significance = {
        gene: ss if gene in top_bottom_genes else 0 for gene, ss in signed_significance
    }

    return extreme_signed_significance


def check_number_of_significant_genes(extreme_signed_significance):
    """Check Number of Significant Genes

    Arguments:
        - extreme_signed_significance (dict): gene_id -> extreme signed significance
    Returns:
        - bool
    """
    values = np.array(list(extreme_signed_significance.values()), dtype=bool)
    if np.sum(values) == 200:
        return True
    else:
        return False


def ExtremeCorLogP_K(p_values, log2fcs, genes, k):
    signed_significance = compute_signed_significance(p_values, log2fcs, genes)
    extreme_signed_significance = compute_extreme_signed_significance(
        signed_significance, k=k
    )

    if check_number_of_significant_genes(extreme_signed_significance):
        return extreme_signed_significance

    return "Error: Nº of significant genes < 200"


def sort_values(labels_1, labels_2, values_2, default_value=0):
    """Sort Values
    Arguments:
        - labels_1 (list): list of values
        - labels_2 (list): list of values
        - values_2 (list): list of values
    Returns:
        - sorted_values (list): sorted list
    """
    d_labels_2_to_values_2 = dict(zip(labels_2, values_2))
    sorted_values = [
        d_labels_2_to_values_2.get(label_1, default_value) for label_1 in labels_1
    ]
    sorted_values = [
        default_value if np.isnan(value) else value for value in sorted_values
    ]

    return sorted_values


def compute_pearson_correlation(data_matrix_1, data_matrix_2=None):
    """Compute Pearson Correlation
    Arguments:
        - matrix (numpy): matrix of values
    Returns:
        - correlation_values (list): correlation list
    """
    # Compute correlation matrix
    if data_matrix_2 is not None:
        correlation_matrix = np.corrcoef(data_matrix_1, data_matrix_2)
    else:
        correlation_matrix = np.corrcoef(data_matrix_1)

    # Extract the upper triangular part of the correlation matrix, excluding the diagonal
    upper_triangular_indices = np.triu_indices_from(correlation_matrix, k=1)
    correlation_values = correlation_matrix[upper_triangular_indices]
    return correlation_values


def iLINCS_ExtremeCorLogP_K(
    ids_1,
    pvalues_by_ids_1,
    log2fcs_by_ids_1,
    genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    log2fcs_by_ids_2=None,
    genes_by_ids_2=None,
    k=100,
):
    if ids_2 is None:
        ess_matrix_1 = list()
        for id_1, pvalues_1, log2fcs_1, genes_1 in zip(
            ids_1, pvalues_by_ids_1, log2fcs_by_ids_1, genes_by_ids_1
        ):
            # get z-scores
            ess_1 = ExtremeCorLogP_K(
                p_values=pvalues_1, log2fcs=log2fcs_1, genes=genes_1, k=k
            )
            ess_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=ess_1,
                default_value=0,
            )
            ess_matrix_1.append(ess_1_sorted)

        ess_matrix_1 = np.array(ess_matrix_1)

        logging.info(f"Matrix Shape: {ess_matrix_1.shape}")

        # compute pearson correlation w/ rest
        r_values = compute_pearson_correlation(
            data_matrix_1=ess_matrix_1, data_matrix_2=None
        )
        return r_values, ess_matrix_1
    else:
        ess_matrix_1 = list()
        for id_1, pvalues_1, log2fcs_1, genes_1 in zip(
            ids_1, pvalues_by_ids_1, log2fcs_by_ids_1, genes_by_ids_1
        ):
            # get z-scores
            ess_1 = ExtremeCorLogP_K(
                p_values=pvalues_1, log2fcs=log2fcs_1, genes=genes_1, k=k
            )
            ess_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=ess_1,
                default_value=0,
            )
            ess_matrix_1.append(ess_1_sorted)

        ess_matrix_1 = np.array(ess_matrix_1)

        ess_matrix_2 = list()
        for id_2, pvalues_2, log2fcs_2, genes_2 in zip(
            ids_2, pvalues_by_ids_2, log2fcs_by_ids_2, genes_by_ids_2
        ):
            # get z-scores
            ess_2 = ExtremeCorLogP_K(
                p_values=pvalues_2, log2fcs=log2fcs_2, genes=genes_2, k=k
            )
            ess_2_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_2,
                values_2=ess_2,
                default_value=0,
            )
            ess_matrix_2.append(ess_1_sorted)

        ess_matrix_2 = np.array(ess_matrix_2)
        logging.info(f"Matrix Shapes: {ess_matrix_1.shape} {ess_matrix_2.shape}")
        # compute pearson correlation w/ rest
        r_values = compute_pearson_correlation(
            data_matrix_1=ess_matrix_1, data_matrix_2=ess_matrix_2
        )
        return r_values

### Gene Set Enrichmente Scores

In [ ]:
"""Gene Set Enrichment Algorithm
"""

### iLINCS: Pearson Correlation Coefficient

In [ ]:
"""Correlation
Pearsons’s correlation between MODZ scores of the two signatures.
"""


def get_z_scores(p_values):
    """Get Z scores
    Arguments:
        - p_values (list): list of p-values
        - genes (list): list of gene ids
    Returns:
        - z_scores (list): list of z-scores
    """

    # Convert to Z-score for a two-tailed test
    z_scores = [norm.ppf(p_value / 2) for p_value in p_values]
    return z_scores


def pearson_correlation(x, y):
    """Pearson Correlation
    Arguments:
        - x (list): list of values
        - y (list): list of values
    Returns:
        - r (float): pearson correlation
    """
    from scipy.stats import pearsonr

    r, p_value = pearsonr(x, y)
    return r


def compute_pearson_correlation(data_matrix_1, data_matrix_2=None):
    """Compute Pearson Correlation
    Arguments:
        - matrix (numpy): matrix of values
    Returns:
        - correlation_values (list): correlation list
    """
    # Compute correlation matrix
    if data_matrix_2 is not None:
        correlation_matrix = np.corrcoef(data_matrix_1, data_matrix_2)
    else:
        correlation_matrix = np.corrcoef(data_matrix_1)

    # Extract the upper triangular part of the correlation matrix, excluding the diagonal
    upper_triangular_indices = np.triu_indices_from(correlation_matrix, k=1)
    correlation_values = correlation_matrix[upper_triangular_indices]
    return correlation_values


def sort_values(labels_1, labels_2, values_2, default_value=0):
    """Sort Values
    Arguments:
        - labels_1 (list): list of values
        - labels_2 (list): list of values
        - values_2 (list): list of values
    Returns:
        - sorted_values (list): sorted list
    """
    d_labels_2_to_values_2 = dict(zip(labels_2, values_2))
    sorted_values = [
        d_labels_2_to_values_2.get(label_1, default_value) for label_1 in labels_1
    ]
    sorted_values = [
        default_value if np.isnan(value) else value for value in sorted_values
    ]

    return sorted_values


def iLINCS_corr(
    ids_1,
    pvalues_by_ids_1,
    genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    genes_by_ids_2=None,
):
    if ids_2 is None:
        z_score_matrix_1 = list()
        for id_1, pvalues_1, genes_1 in zip(ids_1, pvalues_by_ids_1, genes_by_ids_1):
            # get z-scores
            z_scores_1 = get_z_scores(pvalues_1)
            z_scores_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=z_scores_1,
                default_value=0,
            )
            z_score_matrix_1.append(z_scores_1_sorted)

        z_score_matrix_1 = np.array(z_score_matrix_1)

        logging.info(f"Matrix Shape: {z_score_matrix_1.shape}")

        # compute pearson correlation w/ rest
        r_values = compute_pearson_correlation(
            data_matrix_1=z_score_matrix_1, data_matrix_2=None
        )
        return r_values, z_score_matrix_1
    else:
        z_score_matrix_1 = list()
        for id_1, pvalues_1, genes_1 in zip(ids_1, pvalues_by_ids_1, genes_by_ids_1):
            # get z-scores
            z_scores_1 = get_z_scores(pvalues_1)
            z_scores_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=z_scores_1,
                default_value=0,
            )
            z_score_matrix_1.append(z_scores_1_sorted)
        z_score_matrix_1 = np.array(z_score_matrix_1)

        z_score_matrix_2 = list()
        for id_2, pvalues_2, genes_2 in zip(ids_2, pvalues_by_ids_2, genes_by_ids_2):
            # get z-scores
            z_scores_2 = get_z_scores(pvalues_2)
            z_scores_2_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_2,
                values_2=z_scores_2,
                default_value=0,
            )
            z_score_matrix_2.append(z_scores_2_sorted)

        z_score_matrix_2 = np.array(z_score_matrix_2)
        logging.info(
            f"Matrix Shapes: {z_score_matrix_1.shape} {z_score_matrix_2.shape}"
        )
        # compute pearson correlation w/ rest
        r_values = compute_pearson_correlation(
            data_matrix_1=z_score_matrix_1, data_matrix_2=z_score_matrix_2
        )
        return r_values

### iLINCS: Weighted Pearson Correlation Coefficient

In [140]:
"""Correlation
Pearsons’s correlation between MODZ scores of the two signatures.
"""


def get_z_scores(p_values):
    """Get Z scores
    Arguments:
        - p_values (list): list of p-values
        - genes (list): list of gene ids
    Returns:
        - z_scores (list): list of z-scores
    """

    # Convert to Z-score for a two-tailed test
    z_scores = [norm.ppf(p_value / 2) for p_value in p_values]
    return z_scores


def pearson_correlation(x, y):
    """Pearson Correlation
    Arguments:
        - x (list): list of values
        - y (list): list of values
    Returns:
        - r (float): pearson correlation
    """
    from scipy.stats import pearsonr

    r, p_value = pearsonr(x, y)
    return r


def compute_weighted_correlation(x, y, p_values_x, p_values_y):
    """Weighted Correlation"""
    # convert to numpy
    x = np.array(x)
    y = np.array(y)
    p_values_x = np.array(p_values_x)
    p_values_y = np.array(p_values_y)

    weights = np.log(p_values_x * p_values_y)
    mean_x = np.average(x, weights=weights)
    mean_y = np.average(y, weights=weights)
    covariance = np.average((x - mean_x) * (y - mean_y), weights=weights)
    std_x = np.sqrt(np.average((x - mean_x) ** 2, weights=weights))
    std_y = np.sqrt(np.average((y - mean_y) ** 2, weights=weights))
    weighted_corr = covariance / (std_x * std_y)
    return weighted_corr


def compute_matrix_weighted_correlation(
    data_matrix_1, p_values_matrix_1, data_matrix_2=None, p_values_matrix_2=None
):
    """Compute Pearson Correlation
    Arguments:
        - matrix (numpy): matrix of values
    Returns:
        - correlation_values (list): correlation list
    """
    # Compute correlation matrix
    weighted_correlations = list()
    if data_matrix_2 is None:
        len_1 = len(data_matrix_1)
        for i in range(len_1):
            for j in range(
                i + 1, len_1
            ):  # Start from i + 1 to only include upper diagonal
                weighted_correlation = compute_weighted_correlation(
                    data_matrix_1[i],
                    data_matrix_1[j],
                    p_values_matrix_1[i],
                    p_values_matrix_1[j],
                )
                weighted_correlations.append(weighted_correlation)
        return weighted_correlations
    else:
        len_1 = len(data_matrix_1)
        len_2 = len(data_matrix_2)
        for i in range(len_1):
            for j in range(
                i + 1, len_2
            ):  # Start from i + 1 to only include upper diagonal
                weighted_correlation = compute_weighted_correlation(
                    data_matrix_1[i],
                    data_matrix_2[j],
                    p_values_matrix_1[i],
                    p_values_matrix_2[j],
                )
                weighted_correlations.append(weighted_correlation)
        return weighted_correlations


def sort_values(labels_1, labels_2, values_2, default_value=0):
    """Sort Values
    Arguments:
        - labels_1 (list): list of values
        - labels_2 (list): list of values
        - values_2 (list): list of values
    Returns:
        - sorted_values (list): sorted list
    """
    d_labels_2_to_values_2 = dict(zip(labels_2, values_2))
    sorted_values = [
        d_labels_2_to_values_2.get(label_1, default_value) for label_1 in labels_1
    ]
    sorted_values = [
        default_value if np.isnan(value) else value for value in sorted_values
    ]

    return sorted_values


def iLINCS_weighted_corr(
    ids_1,
    pvalues_by_ids_1,
    genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    genes_by_ids_2=None,
):
    if ids_2 is None:
        z_score_matrix_1 = list()
        p_values_matrix_1 = list()
        for id_1, pvalues_1, genes_1 in zip(ids_1, pvalues_by_ids_1, genes_by_ids_1):
            # get z-scores
            z_scores_1 = get_z_scores(pvalues_1)
            z_scores_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=z_scores_1,
                default_value=0,
            )
            p_values_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=pvalues_1,
                default_value=1,
            )
            z_score_matrix_1.append(z_scores_1_sorted)
            p_values_matrix_1.append(p_values_1_sorted)
        z_score_matrix_1 = np.array(z_score_matrix_1)
        p_values_matrix_1 = np.array(p_values_matrix_1)

        logging.info(f"Matrix Shape: {z_score_matrix_1.shape}")

        # compute pearson correlation w/ rest
        r_values = compute_matrix_weighted_correlation(
            data_matrix_1=z_score_matrix_1,
            p_values_matrix_1=p_values_matrix_1,
            data_matrix_2=None,
            p_values_matrix_2=None,
        )
        return r_values, z_score_matrix_1
    else:
        z_score_matrix_1 = list()
        p_values_matrix_1 = list()
        for id_1, pvalues_1, genes_1 in zip(ids_1, pvalues_by_ids_1, genes_by_ids_1):
            # get z-scores
            z_scores_1 = get_z_scores(pvalues_1)
            z_scores_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=z_scores_1,
                default_value=0,
            )
            p_values_1_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_1,
                values_2=pvalues_1,
                default_value=1,
            )
            z_score_matrix_1.append(z_scores_1_sorted)
            p_values_matrix_1.append(p_values_1_sorted)
        z_score_matrix_1 = np.array(z_score_matrix_1)
        p_values_matrix_1 = np.array(p_values_matrix_1)

        z_score_matrix_2 = list()
        p_values_matrix_2 = list()
        for id_2, pvalues_2, genes_2 in zip(ids_2, pvalues_by_ids_2, genes_by_ids_2):
            # get z-scores
            z_scores_2 = get_z_scores(pvalues_2)
            z_scores_2_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_2,
                values_2=z_scores_2,
                default_value=0,
            )
            p_values_2_sorted = sort_values(
                labels_1=human_gene_ids,
                labels_2=genes_2,
                values_2=pvalues_2,
                default_value=1,
            )

            z_score_matrix_2.append(z_scores_2_sorted)
            p_values_matrix_2.append(p_values_2_sorted)

        z_score_matrix_2 = np.array(z_score_matrix_2)
        p_values_matrix_2 = np.array(p_values_matrix_2)
        logging.info(
            f"Matrix Shapes: {z_score_matrix_1.shape} {z_score_matrix_2.shape}"
        )
        # compute pearson correlation w/ rest
        r_values = compute_matrix_weighted_correlation(
            data_matrix_1=z_score_matrix_1,
            p_values_matrix_1=pvalues_by_ids_1,
            data_matrix_2=z_score_matrix_2,
            p_values_matrix_2=pvalues_by_ids_2,
        )
        return r_values

### Spearman's Rank Correlation Coefficient

### Euclidean Distance

### Cosine Similarity

### Extreme Weighted Correlation Top 100 up- & down-

### Connectivity Map

## Loading Data

In [ ]:
def get_DiSignAtlas_signature(dsaid):
    """Open Microarray File
    Arguments:
        - dsaid (str): DiSignAtlas ID
    Returns:
        - genes (list): list of gene ids
        - p_value (list): list of p-values
        - adj_p_value (list): list of adjusted p-values
        - log2fc (list): list of log2fc values
    """
    data_path = os.path.join(
        "..", "data", "DiSignAtlas", "dsa_diff_download", f"{dsaid}_alldiff.txt"
    )

    # open file
    df = pd.read_csv(data_path, sep="\t")

    # retrieve signature data
    p_values = df["PValue"].to_list()
    adj_p_values = df["AdjPValue"].to_list()
    log2fcs = df["Log2FC"].to_list()
    genes = df["GeneID"].to_list()

    return p_values, adj_p_values, log2fcs, genes


def check_microarray_file_path(dsaid):
    """Check Microarray File Path
    Arguments:
        - dsaid: str
    Returns:
        - bool
    """
    data_path = os.path.join(
        "..", "data", "DiSignAtlas", "dsa_diff_download", f"{dsaid}_alldiff.txt"
    )
    return os.path.exists(data_path)


def process_dsaid(dsaid):
    if check_microarray_file_path(dsaid):
        p_values, adj_p_values, log2fcs, genes = get_DiSignAtlas_signature(dsaid)

        return p_values, adj_p_values, log2fcs, genes

    else:
        print("Fuck")
        logging.info(f"{dsaid} no microarray data found")
        return dsaid, None


# 2. Load Data

In [60]:
# Testing Area
dsaid = "DSA00004"
p_values, adj_p_values, log2fcs, genes = process_dsaid(dsaid=dsaid)

logging.info(f"Nº genes {len(genes)} for {dsaid}")

# extreme signed significance {gene_id: ess}
ess = ExtremeCorLogP_K(p_values=adj_p_values, log2fcs=log2fcs, genes=genes, k=100)

2024-01-31 16:32:00,763 - INFO - Nº genes 25607 for DSA00004


In [117]:
dsaids = ["DSA00296", "DSA00297", "DSA00298", "DSA00299", "DSA00029", "DSA00004"]

ids_1 = list()
pvalues_by_ids_1 = list()
adj_pvalues_by_ids_1 = list()
genes_by_ids_1 = list()
log2fcs_by_ids_1 = list()
for dsaid in tqdm(dsaids):
    p_values, adj_p_values, log2fcs, genes = process_dsaid(dsaid=dsaid)
    ids_1.append(dsaid)
    pvalues_by_ids_1.append(p_values)
    adj_pvalues_by_ids_1.append(adj_p_values)
    genes_by_ids_1.append(genes)
    log2fcs_by_ids_1.append(log2fcs)

assert (
    len(pvalues_by_ids_1)
    == len(adj_pvalues_by_ids_1)
    == len(log2fcs_by_ids_1)
    == len(genes_by_ids_1)
    == len(ids_1)
), "Err Nº of elements not correct"

100%|██████████| 6/6 [00:00<00:00, 45.45it/s]


In [130]:
result, a = iLINCS_corr(
    ids_1=ids_1,
    pvalues_by_ids_1=pvalues_by_ids_1,
    genes_by_ids_1=genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    genes_by_ids_2=None,
)

logging.info(f"result: {list(result)}")

2024-02-01 10:28:14,303 - INFO - Matrix Shape: (6, 27338)
2024-02-01 10:28:14,305 - INFO - result: [0.6475252570866299, 0.4850995174417095, 0.3901477955926582, 0.008010595014758017, 0.24681452462867412, 0.6008518564691705, 0.36875200670263875, 0.0031432419309518774, 0.2455904318201574, 0.3720384029381997, -0.000278941633761992, 0.22570505422552398, 0.00860750649330167, 0.2407358986546459, -0.00033619620352616595]


In [131]:
r, m = iLINCS_ExtremeCorLogP_K(
    ids_1=ids_1,
    pvalues_by_ids_1=pvalues_by_ids_1,
    log2fcs_by_ids_1=log2fcs_by_ids_1,
    genes_by_ids_1=genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    log2fcs_by_ids_2=None,
    genes_by_ids_2=None,
    k=100,
)

logging.info(f"result: {list(r)}")

2024-02-01 10:28:16,309 - INFO - Matrix Shape: (6, 27338)
2024-02-01 10:28:16,310 - INFO - result: [1.0, 0.9999999999998835, 0.9999999999998835, -0.0009129582850230039, 0.0022436301561945863, 0.9999999999998835, 0.9999999999998835, -0.0009129582850230039, 0.0022436301561945863, 1.0, -0.0009129582850227731, 0.002243630156194356, -0.0009129582850227731, 0.002243630156194356, 0.006821406384505291]


In [141]:
result, a = iLINCS_weighted_corr(
    ids_1=ids_1,
    pvalues_by_ids_1=pvalues_by_ids_1,
    genes_by_ids_1=genes_by_ids_1,
    ids_2=None,
    pvalues_by_ids_2=None,
    genes_by_ids_2=None,
)

logging.info(f"result: {list(result)}")

2024-02-01 12:39:45,135 - INFO - Matrix Shape: (6, 27338)


TypeError: Axis must be specified when shapes of a and weights differ.

In [123]:
len(a[5])

27338

## Benchmarking